In [1]:
import pandas as pd
import numpy as np
import os
import requests
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.docstore import InMemoryDocstore
import faiss
from typing import List
from langchain.embeddings.base import Embeddings
from tenacity import retry, stop_after_attempt, wait_random_exponential

In [2]:
api_key = os.getenv('API_KEY')

In [3]:
df2 = pd.read_csv('KnowledgeBase - Редкостное дерьмо.csv')
df2.head(10)

,Вопросы,Намерение,Уверенность анализа
0,Почему вы списываете деньги без моего согласия?!,billing,0.98
1,"Где мои проклятые бонусы, которые вы обещали?!",billing,0.95
2,Почему я не могу отключить эту чертову услугу?!,complaint,0.97
3,Кто разрешил вам подключать платные подписки б...,billing,0.99
4,"Почему интернет такой медленный, хотя я плачу ...",technical,0.96
5,Когда вы уже почините эту гребаную связь?!,technical,0.94
6,"Почему мне не приходит СМС с кодом, вы что, из...",technical,0.93
7,"Кто виноват, что у меня сгорел кешбэк?!",billing,0.92
8,"Почему вы не предупредили, что тариф изменится?!",complaint,0.97
9,Зачем вы втихую подключили автопродление?!,billing,0.98


In [ ]:
class intent():
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.api_url = "https://api.gpt.mws.ru/v1/chat/completions"
        
    #@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(200))
    def intent_classifier(self, question):
        headers = {
            "Authorization": f"Bearer {api_key}"  # Replace YOUR_API_KEY with your actual API key
        }
        system_prompt = f"""
        Ты эксперт в области анализа данных, твоя задача внимательно проанализировать запрос пользователя и определить его смысловую нагрузку(намерение)
        Не отвечай на сам вопрос
        Итоговый ответ должен быть на английском языке
        Итоговый ответ должен состоять из одного слова (billing | technical | complaint)
        """
        user_prompt = f"""
            Запрос пользователя: {question}"""
        data = {
            "model": 'deepseek-r1-distill-qwen-32b',
            "messages" : [
                {"role" : "system", "content" : system_prompt},
                {"role" : "user", "content": user_prompt}
            ],
            "temperature": 0.0,
            "n": 1,
            "frequency_penalty": 0,
            "presence_penalty": 0,
            
        }
        
        try:
            response = requests.post(self.api_url, headers=headers, json=data)
            text = response.json()['choices'][0]['message']['content']
            think_end = text.index('</think>')
            start = think_end + len('</think>')
            return text[start:].strip()
        except Exception as e:
            print(e)
            return e

In [ ]:
a = intent(api_key=api_key)
a.intent_classifier("""Последние пару месяцев замечаю, что с баланса пропадают деньги — пишут, что это за какие-то «премиум-сервисы». Я их не активировал, поддержка в прошлый раз сказала, что разберутся, но списания продолжаются.

Как окончательно отключить эти начисления и вернуть списанное?""")

'billing'